In [1]:
import sqlite3
import pandas as pd
from surprise import Dataset, Reader, SVD
from sklearn.model_selection import train_test_split
from surprise import accuracy
import numpy as np


Récupération des données

In [ ]:
db_path = "../../db.sqlite3"  
conn = sqlite3.connect(db_path)

In [4]:
query = """
SELECT 
    main_order.client_id,
    main_pizza.name AS pizza_name
FROM 
    main_order
JOIN 
    main_pizza
ON 
    main_order.pizza_id = main_pizza.pizza_id
"""
data = pd.read_sql_query(query, conn)

In [5]:
client_ids = data['client_id'].unique()

# Split 80/20
train_clients, test_clients = train_test_split(client_ids, test_size=0.2, random_state=42)

train_data = data[data['client_id'].isin(train_clients)]
test_data = data[data['client_id'].isin(test_clients)]

In [6]:
# Calculer le nombre de commandes par client et pizza (fréquence comme score)
train_pizza_counts = train_data.groupby(['client_id', 'pizza_name']).size().reset_index(name='rating')
test_pizza_counts = test_data.groupby(['client_id', 'pizza_name']).size().reset_index(name='rating')

reader = Reader(rating_scale=(train_pizza_counts['rating'].min(), train_pizza_counts['rating'].max()))
trainset = Dataset.load_from_df(train_pizza_counts[['client_id', 'pizza_name', 'rating']], reader).build_full_trainset()
testset = Dataset.load_from_df(test_pizza_counts[['client_id', 'pizza_name', 'rating']], reader).build_full_trainset()

# Entraîner un modèle SVD sur l'ensemble d'entraînement
algo = SVD(n_factors=50, reg_all=0.1)
algo.fit(trainset)

Prédictions ensemble de test

In [7]:
predictions = algo.test(testset.build_testset())

# Calculer les métriques
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


RMSE: 0.8790
MAE:  0.7022
RMSE: 0.8790274778104493
MAE: 0.7021644057586461


In [8]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """
    Calculer la précision et le rappel à k pour une liste de prédictions.

    Args:
    predictions (list of Prediction): Liste des prédictions générées par Surprise.
    k (int): Nombre maximum de recommandations à considérer.
    threshold (float): Seuil au-dessus duquel une prédiction est considérée comme pertinente.

    Returns:
    dict: Dictionnaire contenant la précision et le rappel pour chaque utilisateur.
    """
    # Regrouper les prédictions par utilisateur
    user_est_true = defaultdict(list)
    for pred in predictions:
        user_est_true[pred.uid].append((pred.est, pred.r_ui))  # Note prédite (est), note réelle (r_ui)

    precisions = {}
    recalls = {}

    for uid, user_ratings in user_est_true.items():
        # Trier les prédictions de l'utilisateur par note prédite (descendante)
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # k premières prédictions
        user_ratings = user_ratings[:k]

        # Nombre de prédictions pertinentes
        n_relevant = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Nombre total d'items pertinents pour cet utilisateur
        n_total_relevant = sum((true_r >= threshold) for (_, true_r) in user_est_true[uid])

        precisions[uid] = n_relevant / k if k > 0 else 0
        recalls[uid] = n_relevant / n_total_relevant if n_total_relevant > 0 else 0

    return precisions, recalls

In [9]:
# Précision et rappel
k = 3  # Nb recommandations
threshold = 0.8
precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=threshold)

# Calculer les moyennes globales
avg_precision = sum(precisions.values()) / len(precisions)
avg_recall = sum(recalls.values()) / len(recalls)

print(f"Précision moyenne à {k} : {avg_precision:.4f}")
print(f"Rappel moyen à {k} : {avg_recall:.4f}")

Précision moyenne à 3 : 0.8660
Rappel moyen à 3 : 0.7572


Exemple recommandation client

In [10]:
def test_model_on_existing_client(client_id):
    if client_id not in data['client_id'].unique():
        print(f"Erreur : client_id {client_id} n'existe pas dans les données.")
        return

    # Récupérer l'historique des commandes du client
    client_history = data[data['client_id'] == client_id].groupby('pizza_name').size()
    print(f"Historique des commandes pour le client {client_id} :")
    for pizza, count in client_history.items():
        print(f" - {pizza} (commandée {count} fois)")

    # Considérer toutes les pizzas
    pizza_names = data['pizza_name'].unique()

    # Générer des prédictions pour toutes les pizzas
    recommendations = [algo.predict(client_id, pizza_name) for pizza_name in pizza_names]

    # Trier les recommandations selon score
    top_recommendations = sorted(recommendations, key=lambda x: x.est, reverse=True)[:3]

    print(f"\nTop 3 recommandations pour le client {client_id} :")
    for rec in top_recommendations:
        print(f"Pizza : {rec.iid}, Score estimé : {rec.est}")

# Test
test_client_id = 10 
test_model_on_existing_client(test_client_id)

Historique des commandes pour le client 10 :
 - The Four Cheese Pizza (commandée 1 fois)
 - The Italian Supreme Pizza (commandée 2 fois)
 - The Napolitana Pizza (commandée 1 fois)
 - The Sicilian Pizza (commandée 1 fois)
 - The Spinach Pesto Pizza (commandée 1 fois)
 - The Spinach Supreme Pizza (commandée 1 fois)
 - The Vegetables + Vegetables Pizza (commandée 1 fois)

Top 3 recommandations pour le client 10 :
Pizza : The Barbecue Chicken Pizza, Score estimé : 1.505051873194929
Pizza : The Pepperoni, Mushroom, and Peppers Pizza, Score estimé : 1.4753027805461951
Pizza : The Italian Supreme Pizza, Score estimé : 1.4645881910233374


Sauvegarde du modèle

In [11]:
import pickle

collaborative_model_path = 'collaborative_model.pkl'

with open(collaborative_model_path, 'wb') as f:
    pickle.dump(algo, f)

print(f"Modèle sauvegardé dans {collaborative_model_path}")

Modèle sauvegardé dans collaborative_model.pkl
